In [541]:
from matplotlib import pyplot as plt
from matplotlib.pyplot import triplot
from math import sqrt
import numpy as np
from itertools import combinations
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection, Line3DCollection
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
from enum import IntEnum

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as Func
import torch.optim as optim



from IPython.core.debugger import Tracer
from math import pi,cos,sin
%matplotlib inline

In [542]:
minDihedralAngleThreshold = 30.0
maxDihedralAngleThreshold = 180.0-40.0
minSolidAngleThreshold = 30.0
maxSolidAngleThreshold = 180.0-40.0

class shapeType(IntEnum):
    ROUND=0
    NEEDLE=1
   # WEDGE=2
   # SPINDLE=3
   # SLIVER=4
   # CAP=5
        

def tri_normal(tetra_point_0,tetra_point_1,tetra_point_2):
    A=tetra_point_1-tetra_point_0
    B=tetra_point_2-tetra_point_0
    vector=np.cross(A,B)
    norm=np.linalg.norm(vector)
    return vector/norm
        

def face_normals(tetra):
    tripermutations=[[1,3,2,0],
                     [0,2,3,1],
                     [0,3,1,2],
                     [0,1,2,3]]
    normals=np.empty([4,3])
    for i in range(4):
        normals[i]=tri_normal(tetra[tripermutations[i][0]]
                             ,tetra[tripermutations[i][1]],
                              tetra[tripermutations[i][2]])
    return normals
    
    
def solid_angle(v1,v2,v3):
    matrix=np.array([[v1[0],v1[1],v1[2]],
                   [v2[0],v2[1],v2[2]],
                 [v3[0],v3[1],v3[2]]])
    det=np.linalg.det(matrix)
    v1_norm,v2_norm,v3_norm=np.linalg.norm(v1),np.linalg.norm(v2),np.linalg.norm(v3)
    div=v1_norm*v2_norm*v3_norm+np.dot(v1,v2)*v3_norm+np.dot(v1,v3)*v2_norm+np.dot(v2,v3)*v1_norm
    angle=np.absolute(np.arctan2(det,div))
    angle*=2
    return angle
    
    
def solid_angles(tetra):
    angles=np.empty([4,1])
    for i in range(4):
        a=tetra[(i+1)%4]-tetra[i]
        b=tetra[(i+2)%4]-tetra[i]
        c=tetra[(i+3)%4]-tetra[i]
        angles[i]=solid_angle(a,b,c)
    return angles*180/pi

def dihedral_angles(tetra):
    normals=face_normals(tetra)
    angles=np.empty([6,1])
    counter=0
    for i in range(3):
        for j in range(i+1,4):
            angles[counter]=pi-np.arccos(np.dot(normals[i],normals[j]))
            counter+=1
    return angles*180/pi


def shape_type(tetra):
    
    solidAngles,dihedralAngles=solid_angles(tetra),dihedral_angles(tetra)
    maxSolidangle,minSolidAngle=np.max(solidAngles),np.min(solidAngles)
    maxDihedralangle,minDihedralangle=np.max(dihedralAngles),np.min(dihedralAngles)
    
    smallDihedralAngle=minDihedralangle<minDihedralAngleThreshold
    largeDihedralAngle=maxDihedralangle>maxDihedralAngleThreshold
    smallSolidAngle=minSolidAngle<minSolidAngleThreshold
    largeSolidAngle=maxSolidangle>maxSolidAngleThreshold
    shape=shapeType.ROUND
    
    if not smallDihedralAngle and not largeDihedralAngle:
        if smallSolidAngle: shape=shapeType.NEEDLE
        else:shape=shapeType.ROUND
   
    elif smallDihedralAngle and not largeDihedralAngle:
        shape=shapeType.WEDGE
    
    elif not smallDihedralAngle and largeDihedralAngle and not largeSolidAngle and smallSolidAngle:
        shape=shapeType.SPINDLE
    
    elif smallDihedralAngle and largeDihedralAngle and not largeSolidAngle and smallSolidAngle:
        shape=shapeType.SLIVER
    elif smallDihedralAngle and largeDihedralAngle and  largeSolidAngle and smallSolidAngle:
        shape=shapeType.CAP
    
    return shape.name,shape.value
    

# returns tetrahedron of efdge length 1    
def regular_tet():
    tetrahedron=np.array([[0.57735026918962573,0,0],
                        [-0.28867513459481275, -0.5,0],
                        [-0.28867513459481314, 0.49999999999999983,0],
                        [0,0, 0.81649658092772603]])
    return tetrahedron

# returns tetrahedron of efdge length sqrt(2)  
def squared_regular_tet():
    tetrahedron=np.array([ [1,-1/sqrt(3),-1/sqrt(6)]
                                , [-1,-1/sqrt(3),-1/sqrt(6)] 
                               , [0,2/sqrt(3),-1/sqrt(6)] 
                                   , [0,0,3/sqrt(6)] ])
    return tetrahedron



def get_barycenter(number,pos):
    barycenter=np.zeros([1,3])
    for i in range(number):
        barycenter+=pos[i]
    return barycenter/number

def get_cap_tet(shapefactor):
    tet=regular_tet()
    
    triBarycenter=get_barycenter(3,tet)
    dir2triBarycenter=np.empty([3,3])
    
    for i in range(3):
        dir2triBarycenter[i]=triBarycenter-tet[i]
        length2triBarycenter=np.linalg.norm(dir2triBarycenter[i])
        dir2triBarycenter[i]/=np.linalg.norm(dir2triBarycenter[i])
    
    dir2triBarycenter_opp=triBarycenter-tet[3]
    length2triBarycenter_opp=np.linalg.norm(dir2triBarycenter_opp)
    dir2triBarycenter_opp/=np.linalg.norm(dir2triBarycenter_opp)
    
    dx=shapefactor*length2triBarycenter_opp
    factor=dir2triBarycenter_opp*dx
    for i in range(3):
        tet[3][i]+=factor[0][i]
    return tet
    
def get_needle_tet(shapefactor):
    tet=regular_tet()
    
    tetBarycenter=get_barycenter(4,tet)
    dir2tetBarycenter=np.empty([4,3])
    length2tetBarycenter=np.empty([4,1])

    for i in range(4):
        dir2tetBarycenter[i]=tetBarycenter-tet[i]
        length2tetBarycenter[i]=np.linalg.norm(dir2tetBarycenter[i])
        dir2tetBarycenter[i]/=np.linalg.norm(dir2tetBarycenter[i])
   
    triBarycenter=get_barycenter(3,tet)
    dir2triBarycenter=np.empty([3,3])
    length2triBarycenter=np.empty([3,1])
    for i in range(3):
        dir2triBarycenter[i]=triBarycenter-tet[i]
        length2triBarycenter[i]=np.linalg.norm(dir2triBarycenter[i])
        dir2triBarycenter[i]/=np.linalg.norm(dir2triBarycenter[i])
     
    # COME BACK L 
    dx=shapefactor*np.linalg.norm(triBarycenter-tet[3])
    for i in range(3):
        tet[i]=tet[i]+dir2triBarycenter[i]*dx
    return tet



def get_wedge_tet(shapefactor):
    tet=regular_tet()
    
    segBarycenter=np.empty([2,3])
    segBarycenter[0]=get_barycenter(2,tet)
    segBarycenter[1]=get_barycenter(2,tet[2:])
    
    dir2SegBaryCenter=np.empty([2,2,3])
    length2SegBaryCenter=np.empty([2,3])
    
    for i in range(2):
        dir2SegBaryCenter[0][i]=segBarycenter[0]-tet[i]
        length2SegBaryCenter[0]=np.linalg.norm(dir2SegBaryCenter[0][i])
        dir2SegBaryCenter[0][i]/=np.linalg.norm(dir2SegBaryCenter[0][i])
        
        dir2SegBaryCenter[1][i]=segBarycenter[0]-tet[i+2]
        length2SegBaryCenter[1]=np.linalg.norm(dir2SegBaryCenter[1][i])
        dir2SegBaryCenter[0][i]/=np.linalg.norm(dir2SegBaryCenter[1][i])
        
    dx=shapefactor*length2SegBaryCenter[0]
    for i in range(2):
        tet[i]+=dir2SegBaryCenter[0][i]*dx
    for i in range(2):
        tet[i+2]+=dir2SegBaryCenter[1][i]*dx
    return tet



def get_spindle_tet(shapefactor):
    tet=regular_tet()
    
    segBarycenter=np.empty([2,3])
    segBarycenter[0]=get_barycenter(2,tet)
    segBarycenter[1]=get_barycenter(2,tet[2:])
    
    dir2SegBaryCenter=np.empty([2,2,3])
    length2SegBaryCenter=np.empty([2,3])
    
    for i in range(2):
        dir2SegBaryCenter[0][i]=segBarycenter[0]-tet[i]
        length2SegBaryCenter[0]=np.linalg.norm(dir2SegBaryCenter[0][i])
        dir2SegBaryCenter[0][i]/=np.linalg.norm(dir2SegBaryCenter[0][i])
        
        dir2SegBaryCenter[1][i]=segBarycenter[0]-tet[i+2]
        length2SegBaryCenter[1]=np.linalg.norm(dir2SegBaryCenter[1][i])
        dir2SegBaryCenter[0][i]/=np.linalg.norm(dir2SegBaryCenter[1][i])
        
    seg2segBaryCenter=(segBarycenter[0]+segBarycenter[1])/2
    dirseg2segBaryCenter=np.empty([2,3])
    for i in range(2):
        dirseg2segBaryCenter[i]=seg2segBaryCenter-segBarycenter[i]
        lengthSeg2SegBaryCenter=np.linalg.norm(dirseg2segBaryCenter[i])
        dirseg2segBaryCenter[i]/=np.linalg.norm(dirseg2segBaryCenter[i])
    
    dx=shapefactor*length2SegBaryCenter[0]
    for j in range(2):
        tet[j]+=dir2SegBaryCenter[0][j]*dx
    
    dh=shapefactor*lengthSeg2SegBaryCenter
    for j in range(2):
        tet[j]+=dirseg2segBaryCenter[0]*dh
    
    for j in range(2):
        tet[j+2]+=dirseg2segBaryCenter[1]*dh
    
    return tet



def get_sliver_tet(shapefactor):
    tet=regular_tet()
    
    segBarycenter=np.empty([2,3])
    segBarycenter[0]=get_barycenter(2,tet)
    segBarycenter[1]=get_barycenter(2,tet[2:])
    
    dir2SegBaryCenter=np.empty([2,2,3])
    length2SegBaryCenter=np.empty([2,3])
    
    for i in range(2):
        dir2SegBaryCenter[0][i]=segBarycenter[0]-tet[i]
        length2SegBaryCenter[0]=np.linalg.norm(dir2SegBaryCenter[0][i])
        dir2SegBaryCenter[0][i]/=np.linalg.norm(dir2SegBaryCenter[0][i])
        
        dir2SegBaryCenter[1][i]=segBarycenter[0]-tet[i+2]
        length2SegBaryCenter[1]=np.linalg.norm(dir2SegBaryCenter[1][i])
        dir2SegBaryCenter[0][i]/=np.linalg.norm(dir2SegBaryCenter[1][i])
        
    seg2segBaryCenter=(segBarycenter[0]+segBarycenter[1])/2
    dirseg2segBaryCenter=np.empty([2,3])
    for i in range(2):
        dirseg2segBaryCenter[i]=seg2segBaryCenter-segBarycenter[i]
        lengthSeg2SegBaryCenter=np.linalg.norm(dirseg2segBaryCenter[i])
        dirseg2segBaryCenter[i]/=np.linalg.norm(dirseg2segBaryCenter[i])
    
   
    dh=shapefactor*lengthSeg2SegBaryCenter
    for j in range(2):
        tet[j]+=dirseg2segBaryCenter[0]*dh
    
    for j in range(2):
        tet[j+2]+=dirseg2segBaryCenter[1]*dh
    
    return tet






In [543]:


def rot_x(theta):
    return np.array([[1,0,0],     
                     [0,cos(theta),-sin(theta)],
                      [0,sin(theta),cos(theta)] ])
def rot_y(theta):
    return np.array([[cos(theta),0,sin(theta)],     
                             [0,1,0],
                      [-sin(theta),0,cos(theta)] ])

def rot_z(theta):
    return np.array([[cos(theta),-sin(theta),0],     
                     [sin(theta),cos(theta),0],
                      [0,0,1] ])


def rotate(angle_x,angle_y,angle_z):
    return rot_z(angle_z).dot(rot_y(angle_y)).dot(rot_x(angle_x))

def get_eigenvalues(tetrahedra):
    """
    Computes and returns the deformation gradient
    matrix as well as its eigenvalues
    """
    
    # Reference regular tetrahdra #

    ref_tetrahedra =regular_tet()
    #plt.triplot(ref_triangle[:,0],ref_triangle[:,1],linewidth=7.0)

    # Material coordinates
    dX=np.array([      
       [ref_tetrahedra[1][0]-ref_tetrahedra[0][0] , ref_tetrahedra[2][0]-ref_tetrahedra[0][0],ref_tetrahedra[3][0]-ref_tetrahedra[0][0]] ,   
       [ref_tetrahedra[1][1]-ref_tetrahedra[0][1] , ref_tetrahedra[2][1]-ref_tetrahedra[0][1],ref_tetrahedra[3][1]-ref_tetrahedra[0][1]],
       [ref_tetrahedra[1][2]-ref_tetrahedra[0][2] , ref_tetrahedra[2][2]-ref_tetrahedra[0][2],ref_tetrahedra[3][2]-ref_tetrahedra[0][2]] ,       ])
    
# Material coordinates
    dx=np.array([      
       [tetrahedra[1][0]-tetrahedra[0][0] , tetrahedra[2][0]-tetrahedra[0][0],tetrahedra[3][0]-tetrahedra[0][0]] ,   
       [tetrahedra[1][1]-tetrahedra[0][1] , tetrahedra[2][1]-tetrahedra[0][1],tetrahedra[3][1]-tetrahedra[0][1]],
       [tetrahedra[1][2]-tetrahedra[0][2] , tetrahedra[2][2]-tetrahedra[0][2],tetrahedra[3][2]-tetrahedra[0][2]] ,       ])
    



    
   # The deformation matrix is F=dx*DX^(-1)

    inv_dX=np.linalg.inv(dX)
    F=np.dot(dx,np.linalg.inv(dX))
    #print("Determinant of the deformation gradient is: {0}".format(np.linalg.det(F)))


    # The goal is to find and appropriate diagonalization 
    # of F=U*F'*V.T. To properly diagonalize it we first 
    # consider A=F.T*F. Then by definition (symmetric matrix)
    # the diagonalizarion of A will give always real and non 
    # nonegative singular valures. We have, A=(VF'U.T)(UF'V.T)
    # =VF^2V.T
    
    A=F.T.dot(F)
    
    # Perform an SVD to A we get:
    
    u,s,V_tranpose=np.linalg.svd(A)
    V=V_tranpose.T
    #s,V=np.linalg.eig(A)
    #print("Determinant of eigenvectors V of FF.T is:{0}".format(np.linalg.det(V)))
    
    # The entries of F' are the square roots of s
    F_diag=np.sqrt(s)
    np.sort(F_diag)
    F_diag=np.diag(F_diag)
    
    
    # We compute  U=F*V*(F_diag)^-1
    
    F_diag_inverse=np.linalg.inv(F_diag)
    U=F.dot(V).dot(F_diag_inverse)
    
    # Checking if indeed F=UF'V.T
    #print(np.allclose(F,U.dot(F_diag).dot(V.T)))
    #print(np.linalg.det(U))
    
    # Calculate rotation matrix and calculate rotation angle(z-axis) (Source: http://nghiaho.com/?page_id=846)
    R=np.dot(V,U.T)
    
    # Checking for reflection case
    if np.linalg.det(R)<0:
        V[1,:]*=-1
        R=np.dot(V,U.T)
    
    rotation_angle_x=180/pi*np.arctan2(R[2,1],R[2,2]) #x->(-pi,pi)
    rotation_angle_y=180/pi*np.arctan2(-R[2,0],sqrt(np.power(R[2,1],2)+np.power(R[2,2],2))) #y->(-pi/,pi/2)
    rotation_angle_z=180/pi*np.arctan2(R[1,0],R[0,0]) #z->(-pi/,pi)

    
    
    # now extract the sign of the singular values of F 
    # by sign(u_i*v_i)*F_diag
    signs=[-1 if np.sign(U[:,i].dot(V[:,i]))<0 else 1 for i in range(A.shape[1])]
    eig=[signs[i]*F_diag[i][i] for i in range(A.shape[1])]
    factor=1

    for i in range(A.shape[1]):
        factor*=np.power(F_diag[i,i],1/3)

    # To make the eigenvalues scale invariant multiply them with
    # 1/P(l_i)
    for i in range(len(eig)):
        eig= np.multiply(eig,1/(factor)**(1/3) if factor!=0 else 0)

            
    #print(" Eigenvalues of F are : {0}".format(eig))  
   
     # Treat change of sign due to rotation
    
    ###if  90 < rotation_angle_x <= 180 or -180 <= rotation_angle_x <= -90:
    ###    eig[0]*=-1
    ###    eig[2]*=-1
    ###if  90 < rotation_angle_z <= 180 or -180 <= rotation_angle_z <= -90:
    ###        eig[1]*=-1
    ###        eig[2]*=-1
    ###elif  90 < rotation_angle_z <= 180 or -180 <= rotation_angle_z <= -90:
    ###        eig[0]*=-1
    ###        eig[1]*=-1
    
    return eig,F,U,[rotation_angle_x,rotation_angle_y,rotation_angle_z]

In [544]:


regular_tetrahedron=regular_tet()
normal=face_normals(regular_tetrahedron)

barycenter=get_barycenter(4,regular_tetrahedron)
barycenter
cap_tet=get_cap_tet(1)
needle_tet=get_needle_tet(0.9)
wedge_tet=get_wedge_tet(0.8)
spindle_tet=get_spindle_tet(4)
sliver_tet=get_sliver_tet(1)
dh=dihedral_angles(spindle_tet)
sa=solid_angles(spindle_tet)
dh2=dihedral_angles(sliver_tet)
sa2=solid_angles(sliver_tet)
shape,value=shape_type(cap_tet)
#dh,sa,dh2,sa2
rotating_tetrahedron=np.dot(rotate(pi/3,0,pi/2),regular_tetrahedron.T).T
eig,F,U,rotation_angle=get_eigenvalues(rotating_tetrahedron)
print(eig,rotation_angle)


AttributeError: CAP

In [ ]:
segBarycenter=np.empty([2,3])


In [ ]:
plot_coords_x,plot_coords_y,plot_coords_z=zip(*rotating_tetrahedron)
plot_coords_x2,plot_coords_y2,plot_coords_z2=zip(*regular_tetrahedron)

In [ ]:
#mesh=go.Mesh3d(x=plot_coords_x,y=plot_coords_y,z=plot_coords_z,color='blue',opacity=2)
#mesh2=go.Mesh3d(x=plot_coords_x2,y=plot_coords_y2,z=plot_coords_z2,color='red',opacity=.2)

In [ ]:
#plotly.offline.init_notebook_mode(connected=True)
#plotly.offline.plot([mesh,mesh2])


In [545]:
# building the train data 
nb_of_samples=4000
step=(1.0-0.9)/nb_of_samples
eigen_values=np.empty([2*nb_of_samples,3])
type_values=[]
type_names=[]

#for f in [get_needle_tet,get_sliver_tet,get_wedge_tet,get_cap_tet,get_spindle_tet]:
for f in [get_needle_tet]:
    count=0
    for i in range(nb_of_samples):
        tet=f(0.9+i*step)
        eig,_,_,_=get_eigenvalues(tet)
        eigen_values[i+count*nb_of_samples]=eig
        shape,value=shape_type(tet)
        type_values.append(value)
    
        type_names.append(shape)
    count+=1
    
for i in range(nb_of_samples):
    tet=get_sliver_tet(0.1+i*step) #mainly done to produce round samples(close to regular)
    eigen_values[i+(count)*nb_of_samples]=eig
    shape,value=shape_type(tet)
    type_values.append(value)
    type_names.append(shape)
type_values

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [546]:
eigen_values

array([[ 2.37745348, -0.64855078, -0.64855078],
       [ 2.37724808, -0.6485788 , -0.6485788 ],
       [ 2.37704273, -0.64860681, -0.64860681],
       ..., 
       [ 1.79973476, -0.74541092, -0.74541092],
       [ 1.79973476, -0.74541092, -0.74541092],
       [ 1.79973476, -0.74541092, -0.74541092]])

In [547]:
# building the test data #
nb_of_test_samples=800
step=(1.0-0.9)/nb_of_samples
eigen_values_test=np.empty([2*nb_of_test_samples,3])
type_values_test=[]
type_names_test=[]

#for f in [get_needle_tet,get_sliver_tet,get_wedge_tet,get_cap_tet,get_spindle_tet]:
for f in [get_needle_tet]:

    count=0
    for i in range(nb_of_test_samples):
        tet=f(0.9+i*step)
        eig,_,_,_=get_eigenvalues(tet)
        eigen_values_test[i+count*nb_of_test_samples]=eig
        shape,value=shape_type(tet)
        type_values_test.append(value)
    
        type_names_test.append(shape)
    count+=1
    
for i in range(nb_of_test_samples):
    tet=get_sliver_tet(0.0001+i*step) #mainly done to produce round samples(close to regular)
    eigen_values_test[i+(count)*nb_of_test_samples]=eig
    shape,value=shape_type(tet)
    type_values_test.append(value)
    type_names_test.append(shape)
type_values_test

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [548]:
#manage the training data

X_train=eigen_values
Y_train=np.array(type_values).reshape(2*nb_of_samples) # 6 refers to the number of categories

X_test=eigen_values_test
Y_test=np.array(type_values_test).reshape(2*nb_of_test_samples) # 6 refers to the number of categories


for i in range(2*(nb_of_samples)):
    for j in range(2):
        if np.isinf(X_train[i][j]):
            X_train[i][j]=0
            
for i in range(2*(nb_of_test_samples)):
    for j in range(2):
        if np.isinf(X_test[i][j]):
            X_test[i][j]=0
# Convert to pytorch tennsors

x_tensor=torch.from_numpy(X_train).type(torch.FloatTensor)
x_tensor[x_tensor == float("Inf")] = 0
x_tensor[x_tensor == float("-Inf")] = 0

x_test_tensor=torch.from_numpy(X_train).type(torch.FloatTensor)
x_test_tensor[x_test_tensor == float("Inf")] = 0
x_test_tensor[x_test_tensor == float("-Inf")] = 0



y_tensor=torch.from_numpy(Y_train).type(torch.LongTensor)
y_test_tensor=torch.from_numpy(Y_test).type(torch.LongTensor)
# Convert to pytorch variables
x_variable=Variable(x_tensor,requires_grad=False)
y_variable=Variable(y_tensor,requires_grad=False)



x_test_variable=Variable(x_test_tensor,requires_grad=False)
y_test_variable=Variable(y_test_tensor,requires_grad=False)

# Normalize data
mu,std=x_variable.mean(0),x_variable.std(0)
mu_test,std_test=x_test_variable.mean(0),x_test_variable.std(0)


x_variable.sub_(mu).div_(std)
x_test_variable.sub_(mu).div_(std)
y_variable,x_variable


shuffle=torch.randperm(x_variable.shape[0])
x_variable = x_variable[shuffle]
y_variable=y_variable[shuffle]

In [549]:
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(n_feature, n_hidden)   # hidden layer
        self.fc1_bn=torch.nn.BatchNorm1d(n_hidden)

        self.fc2 = torch.nn.Linear(n_hidden, n_hidden)   # output layer
        self.fc2_bn=torch.nn.BatchNorm1d(n_hidden)
        
        self.fc3 = torch.nn.Linear(n_hidden, n_hidden)   # output layer
        self.fc3_bn=torch.nn.BatchNorm1d(n_hidden)
        
        self.fc4 = torch.nn.Linear(n_hidden, n_hidden)   # output layer
        self.fc4_bn=torch.nn.BatchNorm1d(n_hidden)
        
        self.fc5 = torch.nn.Linear(n_hidden, n_hidden)   # output layer
        self.fc5_bn=torch.nn.BatchNorm1d(n_hidden)
        
        self.fc6 = torch.nn.Linear(n_hidden, n_output)   # output layer


    def forward(self, x):
        x = Func.relu(self.fc1(x))      # activation function for hidden layer
        x=self.fc2_bn(x)

        x = Func.relu(self.fc2(x))
        x=self.fc2_bn(x)
        x =Func.relu(self.fc3(x))
        x=self.fc3_bn(x)
        x = Func.relu(self.fc4(x))
        x=self.fc2_bn(x)
        x =Func.relu(self.fc5(x))
        x=self.fc3_bn(x)
        x =Func.relu(self.fc6(x))

        return x
        #return x

In [558]:
# Set architeture #
net = Net(n_feature=3, n_hidden=100, n_output=2)     # define the network
print(net)  # net architecture

Net(
  (fc1): Linear(in_features=3, out_features=100)
  (fc1_bn): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True)
  (fc2): Linear(in_features=100, out_features=100)
  (fc2_bn): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True)
  (fc3): Linear(in_features=100, out_features=100)
  (fc3_bn): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True)
  (fc4): Linear(in_features=100, out_features=100)
  (fc4_bn): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True)
  (fc5): Linear(in_features=100, out_features=100)
  (fc5_bn): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True)
  (fc6): Linear(in_features=100, out_features=2)
)


In [559]:
#optimizer=torch.optim.Adam(net.parameters(),lr=0.00001)
optimizer = torch.optim.Adam(net.parameters(), lr=1e-7)
#optimizer = torch.optim.SGD(net.parameters(), lr=1e-3,momentum=0.9)
loss_func = torch.nn.CrossEntropyLoss()  # the target label is NOT an one-hotted

In [560]:
# Train the network #

nb_of_epochs=6000

batch_size=100





for t in range(nb_of_epochs):
    sum_loss=0
    for b in range(0,x_variable.size(0),batch_size):
        out = net(x_variable.narrow(0,b,batch_size))                 # input x and predict based on x
        loss = loss_func(out, y_variable.narrow(0,b,batch_size))     # must be (1. nn output, 2. target), the target label is NOT one-hotted
        
        sum_loss+=loss
        optimizer.zero_grad()   # clear gradients for next train
        loss.backward()         # backpropagation, compute gradients
        #print(t,loss.data[0])
        optimizer.step()        # apply gradients
    #print("Epoch:",t,"Loss:",sum_loss.data[0])
    net.eval()
    nb_train_errors=0
    out = net(x_variable)
    values, indices = torch.max(out.data, 1)
    
    for j in range(2*nb_of_samples):
        if indices[j]is not y_variable[j].data[0]:
            nb_train_errors+=1
    nb_test_errors=0
    out_test = net(x_test_variable)
    values_test, indices_test = torch.max(out_test.data, 1)
    for j in range(2*nb_of_test_samples):
        if indices_test[j]is not y_test_variable[j].data[0]:
            nb_test_errors+=1
            
    net.train()
    print("Epoch:",t,"Loss:",sum_loss.data[0],' acc_train_error {}%'.format((100 * nb_train_errors) / (2*nb_of_samples)),
         ' acc_test_error {}%'.format((100 * nb_test_errors) / (2*nb_of_test_samples)))
    
#for t in range(nb_of_epochs):
#    
#    out = net(x_variable)                 # input x and predict based on x
#    loss = loss_func(out, y_variable)     # must be (1. nn output, 2. target), the target label is NOT one-hotted
#        
#       
#    optimizer.zero_grad()   # clear gradients for next train
#    loss.backward()         # backpropagation, compute gradients
#    #print(t,loss.data[0])
#    optimizer.step()        # apply gradients
#    print("Epoch:",t,"Loss:",loss.data[0])

Epoch: 0 Loss: 55.88132095336914  acc_train_error 50.0%  acc_test_error 50.0%
Epoch: 1 Loss: 55.74956130981445  acc_train_error 50.0%  acc_test_error 50.0%
Epoch: 2 Loss: 55.62139129638672  acc_train_error 50.0%  acc_test_error 50.0%
Epoch: 3 Loss: 55.5003776550293  acc_train_error 50.0%  acc_test_error 50.0%
Epoch: 4 Loss: 55.38279342651367  acc_train_error 50.0%  acc_test_error 50.0%
Epoch: 5 Loss: 55.26974868774414  acc_train_error 50.0%  acc_test_error 50.0%
Epoch: 6 Loss: 55.16136932373047  acc_train_error 50.0%  acc_test_error 50.0%
Epoch: 7 Loss: 55.05427169799805  acc_train_error 50.0%  acc_test_error 50.0%
Epoch: 8 Loss: 54.95062255859375  acc_train_error 50.0%  acc_test_error 50.0%
Epoch: 9 Loss: 54.85337829589844  acc_train_error 50.0%  acc_test_error 50.0%
Epoch: 10 Loss: 54.76095199584961  acc_train_error 50.0%  acc_test_error 50.0%
Epoch: 11 Loss: 54.66729736328125  acc_train_error 50.0%  acc_test_error 50.0%
Epoch: 12 Loss: 54.57306671142578  acc_train_error 50.0%  acc_t

In [ ]:
out_test

# 